In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df = pd.read_csv('internships_data_final.csv')
df.dropna(inplace=True)
df.head()

In [ ]:
import ast
df['skills'] = df['skills'].apply(ast.literal_eval)
df.skills = df.skills.apply(' '.join)
df['locations'] = df.locations.str.split(', ').apply(' '.join)

In [17]:
student_profile = ["AutoCAD Adobe Photoshop Networking"]
student_profile_text = "AutoCAD Adobe Photoshop Networking"
vectorizer = TfidfVectorizer(stop_words='english', lowercase=True)
bag = vectorizer.fit_transform(df.skills)
student_profile = vectorizer.transform(student_profile)

In [18]:
similarity_scores = cosine_similarity(student_profile ,bag.toarray())[0]
df["cosine_score"] = similarity_scores
rankings = similarity_scores.argsort()[::-1]

In [23]:
for i in rankings[:5]:
    print(df.skills[i])

AutoCAD Adobe Photoshop Google SketchUp
MySQL Cloud Computing Python SQL Data Analytics Django Machine Learning
Recruitment English Proficiency (Spoken) English Proficiency (Written)
Blogging Digital Marketing Search Engine Optimization (SEO) Creative Writing English Proficiency (Written)
Leadership Sales Support Sales


In [14]:
def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0


In [19]:
# Student skills as a set
student_skills = set(student_profile_text.lower().split())

# Internship skills as sets
df["skills_set"] = df["skills"].apply(lambda x: set(x.lower().split()))


In [20]:
df["jaccard_score"] = df["skills_set"].apply(lambda s: jaccard_similarity(student_skills, s))


In [25]:
# Attach cosine similarity scores to df
df["cosine_score"] = similarity_scores

# Weighted final score
df["final_score"] = 0.7 * df["cosine_score"] + 0.3 * df["jaccard_score"]

# Rank internships
recommendations = df.sort_values(by="final_score", ascending=False).head(5)
recommendations[["skills", "locations"]]


,skills,locations
7,AutoCAD Adobe Photoshop Google SketchUp,Pune Pimpri-Chinchwad
168,Adobe Photoshop Adobe Illustrator,Bhukum
152,Adobe Photoshop Video Editing,Ahmedabad Delhi Indore Pune Surat Mumbai(Hybrid)
366,Adobe Photoshop Adobe Illustrator Adobe Premie...,Pune
182,AutoCAD Adobe Photoshop Google SketchUp Autode...,Pune
